<a href="https://colab.research.google.com/github/shahdhruv29/Big-Data-Processing-Labs/blob/main/labs/Lab02_Map_Reduce_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 2

Dhruv Shah - 202103017

In [ ]:
!pip install --quiet mrjob==0.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Exercise #1

In [ ]:
%%file classification.py
from mrjob.job import MRJob
import math
import csv
import re

def is_float(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

class Classify(MRJob):

    def mapper_init(self):
        # Initialize a dictionary to store class vectors
        self.classes = {}
        i = 0
        with open('/content/gdrive/My Drive/datasets/iris/iris_classes.csv', mode='r') as file:
            for data in csv.reader(file):
                if is_float(data[0]):
                    i += 1
                    data = [float(k) for k in data]
                    self.classes[i] = data

    def mapper(self, key, line):
        # Split the input line into a list of values
        record = re.split(',', line)
        min_distance = math.inf
        cls = 0
        if is_float(record[0]):
            record = [float(i) for i in record]
            for class_no, class_vector in self.classes.items():
                d = 0
                for i in range(len(class_vector)):
                    # Calculate the squared difference for each dimension
                    d += (class_vector[i] - record[i]) ** 2
                if math.sqrt(d) < min_distance:
                    min_distance = math.sqrt(d)
                    cls = class_no
            # Emit the original data vector and its determined class
            yield record, cls

if __name__ == '__main__':
    Classify.run()

Overwriting classification.py


In [ ]:
!python classification.py "/content/gdrive/My Drive/datasets/iris/iris.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/classification.root.20240813.091406.716208
Running step 1 of 1...
job output is in /tmp/classification.root.20240813.091406.716208/output
Streaming final output from /tmp/classification.root.20240813.091406.716208/output...
[6.4, 2.9, 4.3, 1.3]	1
[6.6, 3.0, 4.4, 1.4]	1
[6.8, 2.8, 4.8, 1.4]	1
[6.7, 3.0, 5.0, 1.7]	3
[6.0, 2.9, 4.5, 1.5]	1
[5.7, 2.6, 3.5, 1.0]	1
[5.5, 2.4, 3.8, 1.1]	1
[5.5, 2.4, 3.7, 1.0]	1
[5.8, 2.7, 3.9, 1.2]	1
[6.0, 2.7, 5.1, 1.6]	1
[5.4, 3.0, 4.5, 1.5]	1
[6.0, 3.4, 4.5, 1.6]	1
[6.7, 3.1, 4.7, 1.5]	1
[6.3, 2.3, 4.4, 1.3]	1
[5.6, 3.0, 4.1, 1.3]	1
[5.5, 2.5, 4.0, 1.3]	1
[5.5, 2.6, 4.4, 1.2]	1
[6.1, 3.0, 4.6, 1.4]	1
[5.8, 2.6, 4.0, 1.2]	1
[5.0, 2.3, 3.3, 1.0]	1
[5.6, 2.7, 4.2, 1.3]	1
[5.7, 3.0, 4.2, 1.2]	1
[5.7, 2.9, 4.2, 1.3]	1
[6.2, 2.9, 4.3, 1.3]	1
[5.1, 2.5, 3.0, 1.1]	1
[5.7, 2.8, 4.1, 1.3]	1
[6.3, 3.3, 6.0, 2.5]	3
[5.8, 2.7, 5.1, 1.9]	1
[7.1, 3.0, 

## Exercise#2

Given two files employee "empc.csv" and department "depc.csv", Perform JOIN operation on these two files using the map-reduce approach. Let the joining
condition be "mgr_eno=eno"

In [ ]:
%%file Q2join.py
from mrjob.job import MRJob
import re

smaller_file_loc = "/content/gdrive/My Drive/datasets/mr/depc.csv"

class Q2join(MRJob):
  def mapper_init(self):
      f = open(smaller_file_loc, 'r')
      lines = f.readlines()
      listofsmallfile = {}    #This dictionary is used to store records from the smaller file, with the department number (dno) as the key and the entire record as the value.
      for line in lines:
        record = line.split(',')
        listofsmallfile[record[0]] = record
      self.map = listofsmallfile    #Store the department records dictionary in the 'map' attribute

  def mapper(self, key, line):
    record = re.split(',', line)
    dno = record[6]
    larger_file = self.map.get(dno) # looks up the department number in the listofsmallfile dictionary created in mapper_init to find a corresponding record from the smaller file.
    if larger_file:
      yield dno , str(line) + "  " + str(larger_file)

if __name__ == '__main__':
    Q2join.run()

Overwriting Q2join.py


In [ ]:
!python Q2join.py "/content/gdrive/My Drive/datasets/mr/empc.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/Q2join.root.20240813.092502.242462
Running step 1 of 1...
job output is in /tmp/Q2join.root.20240813.092502.242462/output
Streaming final output from /tmp/Q2join.root.20240813.092502.242462/output...
"1"	"105,James,2027-11-10,M,55000,,1  ['1', 'Headquater', '105', '1971-06-19\\n']"
"4"	"106,Jennifer,1931-06-20,F,43000,105,4  ['4', 'Administration', '106', '1985-01-01\\n']"
"4"	"107,Ahmad,1959-03-29,M,25000,106,4  ['4', 'Administration', '106', '1985-01-01\\n']"
"5"	"101,John,1955-01-09,M,30000,102,5  ['5', 'Research', '102', '1978-05-22\\n']"
"5"	"102,Franklin,1945-12-08,M,40000,105,5  ['5', 'Research', '102', '1978-05-22\\n']"
"4"	"108,Alicia,1958-07-19,F,25000,106,4  ['4', 'Administration', '106', '1985-01-01\\n']"
"5"	"103,Joyce,1962-07-31,F,25000,102,5  ['5', 'Research', '102', '1978-05-22\\n']"
"5"	"104,Ramesh,1952-09-15,M,38000,102,5  ['5', 'Research', '102', '

## Exercise #3

Let you use dataset https://www.kaggle.com/datasets/bobnau/daily-website-visitors and compute
monthly moving average of website visitors (first time and repeat)

In [ ]:
%%file MonthlyMovingAverage.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import pandas as pd

class MonthlyMovingAverage(MRJob):

    def mapper_init(self):
        self.flag = True            # Initialize a flag to skip the first line (header)

    def mapper(self, _, line):
        if self.flag:
            self.flag = False  # Skip the first line (header)
        else:
            record = line.split(",")  # Split the input line into a list of values
            visits_str = record[6].strip().replace('"', '')  # Remove any surrounding whitespace and quotes
            try:
                visits = float(visits_str)  # Attempt to convert the cleaned string to a float
                yield "WebsiteVisits", (int(record[0]), visits)  # Yield the key and value pair
            except ValueError:
                pass  # Skip the record if conversion fails


    def reducer(self, key, value):
        window_size = 30  # Define the size of the rolling window (30 days for monthly moving average)
        rolling_sum = 0   # Initialize the rolling sum
        rolling_window = []  # Initialize the rolling window as an empty list

        for t in value:
            rolling_window.append(t)  # Add the timestamp and visits to the rolling window
            rolling_sum += t[1]      # Update the rolling sum with the new visits

            if len(rolling_window) > window_size:   # If the rolling window exceeds the defined size, remove the oldest element
                rolling_sum -= rolling_window[0][1]  # Subtract the visits of the oldest timestamp
                rolling_window.pop(0)               # Remove the oldest timestamp

            if len(rolling_window) == window_size:                  # If the rolling window has reached the defined size, calculate and yield the average
                yield None, rolling_sum / window_size

if __name__ == '__main__':
    MonthlyMovingAverage.run()

Writing MonthlyMovingAverage.py


In [ ]:
!python MonthlyMovingAverage.py "/content/gdrive/MyDrive/datasets/daily-website-visitors.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MonthlyMovingAverage.root.20240813.051459.832416
Running step 1 of 1...
job output is in /tmp/MonthlyMovingAverage.root.20240813.051459.832416/output
Streaming final output from /tmp/MonthlyMovingAverage.root.20240813.051459.832416/output...
null	1.8
null	1.8666666666666667
null	1.9333333333333333
null	1.9333333333333333
null	1.9333333333333333
null	1.9333333333333333
null	1.9333333333333333
null	1.9333333333333333
null	2.066666666666667
null	2.1333333333333333
null	2.1333333333333333
null	2.066666666666667
null	2.0
null	1.9333333333333333
null	2.0
null	2.1333333333333333
null	2.2
null	2.2
null	2.2666666666666666
null	2.2666666666666666
null	2.2
null	2.2666666666666666
null	2.4
null	2.4
null	2.4
null	2.4
null	2.4
null	2.3333333333333335
null	2.4
null	2.466666666666667
null	2.466666666666667
null	2.466666666666667
null	2.466666666666667
null	2.4
null	2.333333333333333